<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_4_part_2/blob/main/%D0%94%D0%97_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [19]:
import pandas as pd
import  torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

Дані медичне страхування

In [20]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/insurance.csv')

df['sex'] = df['sex'].map({'male': 0, 'female':1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes':1})

df = df.drop(columns=['region'])

data = df[['age', 'bmi', 'expenses']]
df[['age', 'bmi', 'expenses']] = (data - data.min()) / (data.max() - data.min())

In [21]:
df.head()

,age,sex,bmi,children,smoker,expenses
0,0.021739,1,0.320755,0,1,0.251611
1,0.000000,0,0.479784,1,0,0.009636
2,0.217391,0,0.458221,3,0,0.053115
3,0.326087,0,0.180593,0,0,0.333010
4,0.304348,0,0.347709,0,0,0.043816


Потрібно на основі інформації людини спрогнозувати розмір її страховки `expenses`

# Завдання 1
Створіть клас Dataset

In [22]:
class InsuranceDataset(Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df.drop(columns=['expenses']).values, dtype=torch.float32)
        self.y = torch.tensor(df['expenses'].values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = InsuranceDataset(df)

# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [23]:
batch_size = 32

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

X_batch, y_batch = next(iter(dataloader))
print(X_batch.shape)
print(y_batch.shape)


torch.Size([32, 5])
torch.Size([32, 1])


# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-5
* Виберіть різні функції активації: Relu, LeakyRelu, Swish
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [24]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super(InsuranceModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.LeakyReLU(0.01),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        return self.model(x)

model = InsuranceModel()
print(model)

InsuranceModel(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)


# Завдання 4
Визначте функцію втрат та оптимізатор

In [25]:

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [26]:
num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x5 and 4x8)

# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [ ]:
torch.save(model.state_dict(), 'insurance_model.pth')

load_model = InsuranceModel()
load_model.load_state_dict(torch.load('insurance_model.pth'))
with torch.no_grad():
    X_new = torch.tensor([[0.4, 0.3, 0.2, 1.0]], dtype=torch.float32)  # Приклад нових даних
    y_new = load_model(X_new)
    print(f'Predicted Expenses: {y_new.item()}')
